In [1]:
from itertools import product
from dynalearn.util import load_experiments
from dynalearn.util.display import *

save_svg, save_png, save_pdf = False, False, False

/home/charles_murphy/anaconda3/envs/dynalearn_env/lib/python3.7/site-packages/numba/decorators.py:146: RuntimeWarning: Caching is not available when the 'parallel' target is in use. Caching is now being disabled to allow execution to continue.
  warnings.warn(msg, RuntimeWarning)
/home/charles_murphy/anaconda3/envs/dynalearn_env/lib/python3.7/_collections_abc.py:841: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


## Loading data

In [2]:
dynamics = ["sis", "plancksis", "sissis"]
networks = ["ba"]

path = "../../data/case-study/summaries"
exp_names = {d: f"exp-{d}-{networks}" for d in dynamics}
exp = load_experiments(path, exp_names)

dynamics = ["dsir"]
networks = ["w_ba"]

path = "../../data/case-study/summaries"
exp_names = {d: f"exp-{d}-{networks}" for d in dynamics}
exp.update(load_experiments(path, exp_names))

Did not find file `exp-sis-ba.zip`, kept proceding.
Did not find file `exp-plancksis-ba.zip`, kept proceding.


## Making the plot

In [ ]:
config = [("sis", "ba"), ("plancksis", "ba"), ("sissis", "ba"), ("dsir", "w_ba")]
fig, ax = plt.subplots(1, 4, figsize=(4 * 4, 4))

threshold = 10.
dsir_avgk = np.array([1.68, 4.84, 5.92, 7.36, 0.96, 4.52, 5.52, 2.12, 7.6, 6.96]) / 2  
for i, c in enumerate(config):
    x = exp[c].metrics["TrueERSSMetrics"].data["param-absorbing"]
    y = 1 - exp[c].metrics["TrueERSSMetrics"].data["absorbing"][:,0,0]
    yerr = np.sqrt(exp[c].metrics["TrueERSSMetrics"].data["absorbing"][:,1,0])
    index = yerr < threshold
    ax[i].errorbar(x[index], y[index], yerr=yerr[index], marker="o", linestyle="None", color=color_dark["blue"])
    if "epidemic" in exp[c].metrics["TrueERSSMetrics"].data:
        x = exp[c].metrics["TrueERSSMetrics"].data["param-epidemic"]
        y = 1 - exp[c].metrics["TrueERSSMetrics"].data["epidemic"][:,0,0]
        yerr = np.sqrt(exp[c].metrics["TrueERSSMetrics"].data["epidemic"][:,1,0])
        index = yerr < threshold
        ax[i].errorbar(x[index], y[index], yerr=yerr[index], marker="o", linestyle="None", color=color_dark["blue"])
for i, c in enumerate(config):
    x = exp[c].metrics["GNNERSSMetrics"].data["param-absorbing"]
    y = 1 - exp[c].metrics["GNNERSSMetrics"].data["absorbing"][:,0,0]
    yerr = np.sqrt(exp[c].metrics["GNNERSSMetrics"].data["absorbing"][:,1,0])
    index = yerr < threshold
    ax[i].errorbar(x[index], y[index], yerr=yerr[index], marker="^", linestyle="None", color=color_dark["orange"])
    if "epidemic" in exp[c].metrics["GNNERSSMetrics"].data:
        x = exp[c].metrics["GNNERSSMetrics"].data["param-epidemic"]
        y = 1 - exp[c].metrics["GNNERSSMetrics"].data["epidemic"][:,0,0]
        yerr = np.sqrt(exp[c].metrics["GNNERSSMetrics"].data["epidemic"][:,1,0])
        index = yerr < threshold
        ax[i].errorbar(x[index], y[index], yerr=yerr[index], marker="^", linestyle="None", color=color_dark["orange"])
    if c[0] != "dsir":
        ax[i].axvline(4, linestyle="--", color=color_dark["grey"], linewidth=2)
    else:
        for k in dsir_avgk:
            ax[i].axvline(k, linestyle="--", color=color_dark["grey"], linewidth=2)
    ax[i].set_xlabel(r"Average degree [$\langle k \rangle$]", fontsize=large_fontsize)
    ax[i].set_ylabel(r"Prevalence", fontsize=large_fontsize)
    ax[i].set_xlim([x.min(), x.max()])
    ax[i].set_ylim([-0.1, 1.1])
ax[-1].set_ylabel(r"Outbreak size", fontsize=large_fontsize)
# ax[0].set_xlim([x.min(), x.max()])
# ax[1].set_xlim([x.min(), x.max()])
# ax[2].set_xlim([4, 7])

ax[0].set_title(r"\textbf{Simple}", fontsize=large_fontsize)
ax[1].set_title(r"\textbf{Complex}", fontsize=large_fontsize)
ax[2].set_title(r"\textbf{Interacting}", fontsize=large_fontsize)
ax[3].set_title(r"\textbf{Metapopulation}", fontsize=large_fontsize)

label_plot(ax[0], r"\textbf{(a)}", loc="upper left")
label_plot(ax[1], r"\textbf{(b)}", loc="upper left")
label_plot(ax[2], r"\textbf{(c)}", loc="upper left")
label_plot(ax[3], r"\textbf{(d)}", loc="upper left")
    
    
handles = []
handles.append(
    Line2D(
            [-1], [-1], 
            linestyle="--", 
            marker="None", 
            linewidth=2,
            markersize=8,
            color=color_dark["grey"],
            label=r"Training $\langle k \rangle$"
    )
)
handles.append(
    Line2D(
            [-1], [-1], 
            linestyle="-", 
            marker="o", 
            linewidth=3,
            markersize=8,
            color=color_dark["blue"], 
            label=r"GT",
    )
)
handles.append(
    Line2D(
            [-1], [-1], 
            linestyle="-", 
            marker="^", 
            linewidth=3,
            markersize=8,
            color=color_dark["orange"],
            label=r"GNN"
    )
)
ax[0].legend(
    handles=handles, loc="center left", fancybox=False, fontsize=16, framealpha=0., ncol=1, handletextpad=0.1
)
    

plt.tight_layout(w_pad=1)

figname = "manuscript-figure3"
if save_png:
    fig.savefig(os.path.join("png", f"{figname}.png"))
if save_pdf:
    fig.savefig(os.path.join("pdf", f"{figname}.pdf"))
if save_svg:
    fig.savefig(os.path.join("svg", f"{figname}.svg"))